In [ ]:
import gym
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt

In [ ]:
env = gym.make("FrozenLake-v1")
input_size = env.observation_space.n
output_size = env.action_space.n
discount_rate = 0.99
num_episodes = 1000
r_list = []

In [ ]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Flatten(input_shape=(1, 16)))
model.add(tf.keras.layers.Dense(32, activation="relu"))
model.add(tf.keras.layers.Dense(16, activation="relu"))
model.add(tf.keras.layers.Dense(output_size, activation="softmax"))
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
model.compile(
    loss = "mse",
    optimizer = optimizer,
    metrics = ["mae", "mse"]
)
model.summary()

In [ ]:
def onehot(s_no):
    ret = np.zeros((1, 16), dtype=np.float32)
    ret[0, s_no] = 1
    ret = np.expand_dims(ret, 1)
    return ret

In [ ]:
action_cnt_list = []
for i in range(0, num_episodes): #에피소드가 진행되면서 model도 계속 업데이트 된다!!!
    s = env.reset()
    e = 1. / ((i / 50) +10)
    r_all = 0
    done = False
    local_loss = []

    action_cnt = 0
    while not done:
        action_cnt += 1
        print("에피소드 : {}/{} ({}%), action_cnt : {}  ".format(i+1, num_episodes, (100*(i+1))/num_episodes, action_cnt), end="\r")
        Qs = model.predict(onehot(s)) #전혀 훈련되지 않은 채로 일단 predict
        #Qs : [0.12, 0.29, 0.01, 0.05]
        if np.random.rand(1) < e:
            a = env.action_space.sample()
        else:
            a = np.argmax(Qs) #4개 액션 중 가장 큰 값을 가진 녀석의 index no를 알려줄 것
        
        s1, reward, done, _ = env.step(a) #이동해 본다.
        if done: #목적지에 다다르면
            Qs[0, a] = reward #predict를 통해 나온 4개 액션에 reward를 입력 -> Y라벨을 업데이트 하는 것
        else:
            Qs1 = model.predict(onehot(s1)) #현재 state를 input으로 하여 다시 predict를 해서 다음 action을 구하는 것
            Qs[0, a] = reward + discount_rate*np.max(Qs1) #predict를 통해 나온 4개 액션에 reward를 입력하는 것 -> Y라벨을 업데이트 하는 것
        
        model.fit(x=onehot(s), y=Qs, verbose=False)
        r_all += reward #이번 에피소드의 총보상값 업데이트
        s = s1
    r_list.append(r_all)
    action_cnt_list.append(action_cnt)

print("Percent of successful episode: {}".format((100*sum(r_list))/num_episodes)+"%")
plt.bar(range(len(r_list)), r_list, color='blue')
plt.show()

action_cnt_df = pd.DataFrame(action_cnt_list)
action_cnt_df